Star Schema Structure (Pharma Manufacturing Domain)

Fact Tables:

Table Name--------Source Table---------Description

fact_production---silver_production----Each manufacturing batch

fact_sales--------silver_sale----------Regional product sales

fact_qc-----------silver_qc_results----Quality test results per batch

Dimension Tables:

Table Name-------Source Table------------Description

dim_product------Derived from facts------Unique products from production/sales

dim_supplies-----silver_suppliers--------Supplier reference

dim_region-------silver_sales------------Unique region list

dim_machine------silver_iot_metrics------Machine IDs and metadata

#create dim_product

In [1]:
from pyspark.sql.functions import *
from pyspark.sql import *

StatementMeta(, 501caa92-8923-4e82-afab-a1964562dd3f, 3, Finished, Available, Finished)

In [2]:
df_prod = spark.table("PharmaLakehouse.silver_production")
df_sales = spark.table("PharmaLakehouse.silver_sales")
display(df_prod.limit(5))
display(df_sales.limit(5))

StatementMeta(, 501caa92-8923-4e82-afab-a1964562dd3f, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 7b28e84e-9a8c-4927-8462-c18987039de3)

SynapseWidget(Synapse.DataFrame, 83a323ba-bb31-4cd5-9e9e-91c1fd3aa87b)

In [3]:
df_products = df_prod.select("product_id").union(df_sales.select("product_id")).dropDuplicates()
df_products = df_products.withColumn("product_key", expr("monotonically_increasing_id()"))
display(df_products)


StatementMeta(, 501caa92-8923-4e82-afab-a1964562dd3f, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, fb37c350-b194-4f11-89ef-21796f476612)

In [4]:
df_products.write.mode("overwrite").saveAsTable("gold_dim_product")

StatementMeta(, 501caa92-8923-4e82-afab-a1964562dd3f, 6, Finished, Available, Finished)

In [10]:
dim_sup = spark.table("PharmaLakehouse.silver_suppliers")
display(dim_sup)

StatementMeta(, 501caa92-8923-4e82-afab-a1964562dd3f, 12, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 56a92ffa-aa58-489c-a363-e918396dc333)

In [11]:
dim_sup = dim_sup.withColumn("supplier_key",expr("monotonically_increasing_id()"))
dim_sup = dim_sup.write.format("delta").mode("overwrite").saveAsTable("gold_dim_suppliers")

StatementMeta(, 501caa92-8923-4e82-afab-a1964562dd3f, 13, Finished, Available, Finished)

In [14]:
df = spark.table("PharmaLakehouse.gold_dim_suppliers")
display(df)

StatementMeta(, 501caa92-8923-4e82-afab-a1964562dd3f, 16, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a8fe40cc-baed-4286-9358-1c47dd302443)

In [16]:
dim_region = spark.table("PharmaLakehouse.silver_sales")
display(dim_region.limit(5))


StatementMeta(, 501caa92-8923-4e82-afab-a1964562dd3f, 18, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a6203477-33a6-4303-8531-6bd8ccd0a7b0)

In [19]:
dim_region = dim_region.select("region").dropDuplicates().withColumn("region_key",expr("monotonically_increasing_id()"))
dim_region.write.format("delta").mode("overwrite").saveAsTable("gold_dim_region")

StatementMeta(, 501caa92-8923-4e82-afab-a1964562dd3f, 21, Finished, Available, Finished)

In [24]:
dim_machine = spark.table("PharmaLakehouse.silver_iot_metrics")
display(dim_machine.limit(5))

StatementMeta(, 501caa92-8923-4e82-afab-a1964562dd3f, 26, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, de408680-bedc-45e4-adea-811bb065e110)

In [25]:
dim_machine = dim_machine.select("machine_id").dropDuplicates().withColumn("machine_key", expr("monotonically_increasing_id()"))
display(dim_machine)

StatementMeta(, 501caa92-8923-4e82-afab-a1964562dd3f, 27, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f8d50a79-cfc5-4956-81ce-894f43040e9e)

In [26]:
dim_machine.write.mode("overwrite").saveAsTable("gold_dim_machine")

StatementMeta(, 501caa92-8923-4e82-afab-a1964562dd3f, 28, Finished, Available, Finished)

#Creating Fact Tables

In [32]:
fact_prod = spark.table("PharmaLakehouse.silver_production")
display(fact_prod.limit(5))

StatementMeta(, 501caa92-8923-4e82-afab-a1964562dd3f, 34, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, fd56e60c-c832-471c-b765-08a8d5b9ef19)

In [33]:
fact_prod.write.mode("overwrite").saveAsTable("gold_fact_product")

StatementMeta(, 501caa92-8923-4e82-afab-a1964562dd3f, 35, Finished, Available, Finished)

In [36]:
fact_sales = spark.table("PharmaLakehouse.silver_sales")
fact_sales.write.mode("overwrite").saveAsTable("gold_fact_sales")

StatementMeta(, 501caa92-8923-4e82-afab-a1964562dd3f, 38, Finished, Available, Finished)

In [37]:
fact_qc = spark.table("PharmaLakehouse.silver_qc_reports")
fact_qc.write.mode("overwrite").saveAsTable("gold_fact_qc")

StatementMeta(, 501caa92-8923-4e82-afab-a1964562dd3f, 39, Finished, Available, Finished)